In [1]:
import digitalhub as dh

In [2]:
project = dh.get_or_create_project("project-dbt")

In [3]:
project

{'kind': 'project', 'metadata': {'project': 'project-dbt', 'name': 'project-dbt', 'created': '2024-10-22T12:32:14.281Z', 'updated': '2024-10-22T12:32:14.281Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'context': './', 'functions': [], 'artifacts': [], 'workflows': [], 'dataitems': [], 'models': []}, 'status': {'state': 'CREATED'}, 'user': 'khurshid@fbk.eu', 'id': 'project-dbt', 'name': 'project-dbt', 'key': 'store://project-dbt'}

In [4]:
print(project)

{'kind': 'project', 'metadata': {'project': 'project-dbt', 'name': 'project-dbt', 'created': '2024-10-22T12:32:14.281Z', 'updated': '2024-10-22T12:32:14.281Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'context': './', 'functions': [], 'artifacts': [], 'workflows': [], 'dataitems': [], 'models': []}, 'status': {'state': 'CREATED'}, 'user': 'khurshid@fbk.eu', 'id': 'project-dbt', 'name': 'project-dbt', 'key': 'store://project-dbt'}


In [14]:
url = "https://gist.githubusercontent.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv"
di = project.new_dataitem(name="employees-dbt",
                          kind="table",
                          path=url)

In [15]:
sql = """
WITH tab AS (
    SELECT  *
    FROM    {{ ref('employees') }}
)
SELECT  *
FROM    tab
WHERE   tab."DEPARTMENT_ID" = '50'
"""

In [16]:
function = project.new_function(name="function-dbt",
                                kind="dbt",
                                code=sql)

In [17]:
run = function.run("transform",
                   inputs={"employees": di.key},
                   outputs={"output_table": "department-50"})

In [18]:
print(run.refresh().status)

{'state': 'RUNNING', 'message': 'job j-dbttransform-f1a0e3c3-42b2-4559-9a77-4930d5957f1f created', 'transitions': [{'status': 'RUNNING', 'message': 'job j-dbttransform-f1a0e3c3-42b2-4559-9a77-4930d5957f1f created', 'time': '2024-10-22T12:33:14.360687869Z'}, {'status': 'READY', 'time': '2024-10-22T12:33:14.32365682Z'}, {'status': 'BUILT', 'time': '2024-10-22T12:33:14.30845165Z'}], 'k8s': {'secret': {'secret': {'apiVersion': 'v1', 'kind': 'Secret', 'metadata': {'labels': {'dhcore/framework': 'k8sjob', 'app.kubernetes.io/managed-by': 'dhcore', 'app.kubernetes.io/part-of': 'dhcore-project-dbt', 'dhcore/project': 'project-dbt', 'dhcore/runtime': 'dbt', 'app.kubernetes.io/instance': 'dhcore-f1a0e3c3-42b2-4559-9a77-4930d5957f1f', 'app.kubernetes.io/version': 'f1a0e3c3-42b2-4559-9a77-4930d5957f1f'}, 'name': 'sec-dbttransform-f1a0e3c3-42b2-4559-9a77-4930d5957f1f', 'namespace': 'digitalhub-tenant2'}}}, 'job': {'job': {'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'creationTimestamp': 172

In [10]:
print(run)

{'kind': 'dbt+run', 'metadata': {'project': 'project-dbt', 'name': '802f80c4-d1b9-423e-abc6-3931079dece4', 'created': '2024-10-22T12:32:17.488Z', 'updated': '2024-10-22T12:32:17.547Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'dbt+transform://project-dbt/function-dbt-nk:41d32640-9a72-4c2c-80a1-dbb1aeca55ba', 'local_execution': False, 'function': 'dbt://project-dbt/function-dbt-nk:41d32640-9a72-4c2c-80a1-dbb1aeca55ba', 'source': {'base64': 'CldJVEggdGFiIEFTICgKICAgIFNFTEVDVCAgKgogICAgRlJPTSAgICB7eyByZWYoJ2VtcGxveWVlcycpIH19CikKU0VMRUNUICAqCkZST00gICAgdGFiCldIRVJFICAgdGFiLiJERVBBUlRNRU5UX0lEIiA9ICc1MCcK', 'lang': 'sql'}, 'inputs': {'employees': 'store://project-dbt/dataitem/table/employees-dbt-nk:ef6cfb1a-c0ce-4d81-acee-46825f149f74'}, 'outputs': {'output_table': 'department-50'}, 'parameters': {}}, 'status': {'state': 'RUNNING', 'message': 'job j-dbttransform-802f80c4-d1b9-423e-abc6-3931079dece4 created', 'transitions': [{'status': 'RUNNING', '

In [19]:
run

{'kind': 'dbt+run', 'metadata': {'project': 'project-dbt', 'name': 'f1a0e3c3-42b2-4559-9a77-4930d5957f1f', 'created': '2024-10-22T12:33:14.301Z', 'updated': '2024-10-22T12:33:14.364Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'dbt+transform://project-dbt/function-dbt:3923eafb-2c33-4445-b40d-d105399a1279', 'local_execution': False, 'function': 'dbt://project-dbt/function-dbt:3923eafb-2c33-4445-b40d-d105399a1279', 'source': {'base64': 'CldJVEggdGFiIEFTICgKICAgIFNFTEVDVCAgKgogICAgRlJPTSAgICB7eyByZWYoJ2VtcGxveWVlcycpIH19CikKU0VMRUNUICAqCkZST00gICAgdGFiCldIRVJFICAgdGFiLiJERVBBUlRNRU5UX0lEIiA9ICc1MCcK', 'lang': 'sql'}, 'inputs': {'employees': 'store://project-dbt/dataitem/table/employees-dbt:cd94b87c-c8dd-4b9f-9186-1ded416329d5'}, 'outputs': {'output_table': 'department-50'}, 'parameters': {}}, 'status': {'state': 'RUNNING', 'message': 'job j-dbttransform-f1a0e3c3-42b2-4559-9a77-4930d5957f1f created', 'transitions': [{'status': 'RUNNING', 'message':

In [20]:
proj = dh.get_or_create_project("project-dbt")
di = proj.get_dataitem('department-50')
df = di.as_df()
df.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50
1,199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50
2,120,Matthew,Weiss,MWEISS,650.123.1234,18-JUL-04,ST_MAN,8000,-,100,50
3,121,Adam,Fripp,AFRIPP,650.123.2234,10-APR-05,ST_MAN,8200,-,100,50
4,122,Payam,Kaufling,PKAUFLIN,650.123.3234,01-MAY-03,ST_MAN,7900,-,100,50
